In 04_CNN, we loaded a pretrained model, extracted features upfront using it 
and then fed the precomputed features to a FFN and train the FFN.

In this one, we do feature extraction on the fly by adding layers to the pretrained model. Train only the new layers

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Load the model

In [7]:
from keras.applications import VGG16

In [8]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(150,150,3))

In [9]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

### Add layers to this model

In [11]:
from keras import Sequential
from keras.layers import Dense, Flatten

In [12]:
model = Sequential()

model.add(base_model)
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [14]:
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

('This is the number of trainable weights before freezing the conv base:', 30)


In [15]:
base_model.trainable = False

print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

('This is the number of trainable weights before freezing the conv base:', 4)


In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [16]:
model.trainable_weights

[<tf.Variable 'dense_1/kernel:0' shape=(8192, 256) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>]

### Set up data sources

In [17]:
import os

In [18]:
root_dir = "/"
users = os.path.join(root_dir, "Users")
airwoot = os.path.join(users, "airwoot")
documents = os.path.join(airwoot, "Documents")
anuj = os.path.join(documents, "Anuj")
Warehouse = os.path.join(anuj, "Warehouse")


src_folder = os.path.join(Warehouse, "train")

cat_src_folder = os.path.join(src_folder, "CAT")
dog_src_folder = os.path.join(src_folder, "DOG")

dest_data_folder = os.path.join(Warehouse, "cats_and_dogs")

train_dir = os.path.join(dest_data_folder, "train")
cat_train_dir = os.path.join(train_dir, "cat")
dog_train_dir = os.path.join(train_dir, "dog")


test_dir = os.path.join(dest_data_folder, "test")
cat_test_dir = os.path.join(test_dir, "cat")
dog_test_dir = os.path.join(test_dir, "dog")


validation_dir = os.path.join(dest_data_folder, "validation")
cat_validation_dir = os.path.join(validation_dir, "cat")
dog_validation_dir = os.path.join(validation_dir, "dog")

In [21]:
### sanity checks

print(len(os.listdir(cat_train_dir)))
print(len(os.listdir(dog_train_dir)))

print(len(os.listdir(cat_test_dir)))
print(len(os.listdir(dog_test_dir)))

print(len(os.listdir(cat_validation_dir)))
print(len(os.listdir(dog_validation_dir)))

2000
2000
1000
1000
1000
1000


In [19]:
### set up data streams

In [20]:
from keras.preprocessing.image import ImageDataGenerator

In [24]:
train_IDG = ImageDataGenerator(rescale=1.0/255,
                               horizontal_flip = True,
                               vertical_flip = True)

train_DG = train_IDG.flow_from_directory(train_dir, 
                                         target_size=(150, 150), 
                                         batch_size = 20,
                                         class_mode='binary')

Found 4000 images belonging to 2 classes.


In [25]:
validation_IDG = ImageDataGenerator(rescale=1.0/255)

validation_DG = validation_IDG.flow_from_directory(validation_dir, 
                                         target_size=(150, 150), 
                                         batch_size = 20,
                                         class_mode='binary')

Found 2000 images belonging to 2 classes.


### Train it

In [26]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [28]:
history = model.fit_generator(steps_per_epoch=5,epochs=2,
                              generator=train_DG, validation_data=validation_DG, )

Epoch 1/2
4/5 [=======================>......] - ETA: 2:31 - loss: 1.2582 - acc: 0.6250

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
    
plt.show()